In [ ]:
import numpy as np
import pypoman
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from plotting import generate_polygon, make_rectangle
from module import *

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
collapses = 0

parameters_realisations = []
theta_x = 0.5 
theta_y_s = np.linspace(0.501, 0.999, 20)


for i, theta_y in enumerate(theta_y_s):
    
    # Get parameters for the marginal probabilities of the causes 
    # and the conditional of the effects given the causes.
    theta_x, theta_y, theta_z_00, theta_z_01, theta_z_10, theta_z_11, theta_z = generate_instance_and_model(theta_x, theta_y)

    parameters = {}
    
    parameters["theta_x"] = theta_x
    parameters["theta_y"] = theta_y
    parameters["theta_z_00"] = theta_z_00
    parameters["theta_z_01"] = theta_z_01
    parameters["theta_z_10"] = theta_z_10
    parameters["theta_z_11"] = theta_z_11

    parameters_realisations.append(parameters)
    
    # Get the vectors alpha, beta with the marginal distributions P_{XZ} and P_{YZ}
    alpha = get_alpha(theta_x, theta_y, theta_z_00, theta_z_01, theta_z_10, theta_z_11)
    beta = get_beta(theta_x, theta_y, theta_z_00, theta_z_01, theta_z_10, theta_z_11)
    
    # Get "trivial" marginal bounds---lambda_x_min, lambda_x_max, lambda_y_min, lambda_y_max
    lambda_x_min, lambda_x_max, a_0 = get_trivial_bounds(alpha)
    lambda_y_min, lambda_y_max, b_0 = get_trivial_bounds(beta)

    # Get the constraint matrices from the marginal probabilities
    A, B = get_constraint_matrices(theta_x, theta_y)
    
    # Get the bounds from structural causal marginal problem through linear program
    LB_X, UB_X, LB_Y, UB_Y = get_linear_prog_bounds(a_0, b_0, A, B, lambda_x_min, lambda_x_max, lambda_y_min, lambda_y_max)
    
    parameters["lambda_x_min"]= lambda_x_min
    parameters["lambda_x_max"]= lambda_x_max
    parameters["lambda_y_min"]= lambda_y_min
    parameters["lambda_y_max"]= lambda_y_max
    parameters["LB_X"]= LB_X
    parameters["UB_X"]= UB_X
    parameters["LB_Y"]= LB_Y
    parameters["UB_Y"]= UB_Y

In [ ]:
# import os
# figure_path = ''

In [ ]:
def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique), loc='upper center', bbox_to_anchor=(0.5, 1.25))

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))

plt.rcParams['font.size'] = '12'
    
for parameter in parameters_realisations:
    lambda_x_min, lambda_x_max = parameter['lambda_x_min'], parameter['lambda_x_max']
    lambda_y_min, lambda_y_max = parameter['lambda_y_min'], parameter['lambda_y_max']
    LB_X, UB_X = parameter['LB_X'], parameter['UB_X']
    LB_Y, UB_Y = parameter['LB_Y'], parameter['UB_Y']
    ax.hlines(y=lambda_y_min, xmin=lambda_x_min, xmax=lambda_x_max, linewidth=5, color='#be6666', alpha=0.7)
    ax.plot([LB_X, lambda_x_max], [lambda_y_min, lambda_y_min], 's', markersize=3, color='#016ddb')
    ax.hlines(y=lambda_y_min, xmin=LB_X, xmax=lambda_x_max, linewidth=1.5, linestyle='solid', color='white', alpha=1.0)
    ax.hlines(y=lambda_y_min, xmin=LB_X, xmax=lambda_x_max, linewidth=1.5, linestyle='solid', color='#016ddb', alpha=1.0)
xlabel = '$\lambda_X$'
ylabel = '$\lambda_Y$'
plt.xlabel(xlabel, fontsize=15, labelpad=5)
plt.ylabel(ylabel, fontsize=15, rotation=0, labelpad=20)

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(12)
plt.gca().set_aspect('equal', adjustable='box')
# plt.savefig(os.path.join(figure_path, 'figure_3_a.pdf'), dpi=None, facecolor='w', edgecolor='w',
#             orientation='portrait', format=None,
#             transparent=True, bbox_inches='tight', pad_inches=0.1, metadata=None)
plt.show()